# Paper Tables and Figures
- Goal: Build publication-ready summary tables from saved experiment outputs.
- Flow: Load result files, compute formatted summaries, and export tables.


## Load Results
- Step: Read required simulation and application summary files.
- Check: Validate expected schema before table construction.


In [ ]:
import pandas as pd
from pathlib import Path
from IPython.display import display

RESULTS_ROOT = Path('../results')
SIM_ROOT = RESULTS_ROOT / 'simulations'
APP_ROOT = RESULTS_ROOT / 'application'
APP_SUMMARY = APP_ROOT / 'summary'

print('RESULTS_ROOT:', RESULTS_ROOT.resolve())

In [ ]:
TABLE_PATHS = {
    # simulations
    'sim_summary_ab_long': SIM_ROOT / 'summary_ALL.csv',
    'sim_summary_ab_wide': SIM_ROOT / 'summary_ALL_wide.csv',
    'sim_summary_cd_long': SIM_ROOT / 'summary_ALL_CD.csv',
    'sim_placebo_ab_pivot': SIM_ROOT / 'summary_placebo_table.csv',
    'sim_placebo_cd_pivot': SIM_ROOT / 'summary_placebo_table_CD.csv',

    # application
    'app_taskA_all_categories': APP_SUMMARY / 'taskA_model_metrics_all_categories.csv',
    'app_taskA_category_summary': APP_SUMMARY / 'taskA_category_summary.csv',
    'app_taskB_te_all_categories': APP_SUMMARY / 'taskB_placebo_te_summary_all_categories.csv',
    'app_taskB_split_all_categories': APP_SUMMARY / 'taskB_placebo_split_metrics_all_categories.csv',
}

for k, v in TABLE_PATHS.items():
    print(f'{k:30s} -> {v}')

In [ ]:
def load_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(path)
    return pd.read_csv(path)

TABLES = {name: load_csv(path) for name, path in TABLE_PATHS.items()}

status_rows = []
for name, df in TABLES.items():
    status_rows.append({
        'table': name,
        'rows': int(df.shape[0]),
        'cols': int(df.shape[1]),
        'columns': ', '.join(df.columns.astype(str).tolist()),
    })

status_df = pd.DataFrame(status_rows).sort_values('table').reset_index(drop=True)
display(status_df)

In [ ]:
# Quick preview
for name in sorted(TABLES):
    print(f'\n=== {name} ===')
    display(TABLES[name].head(3))

## Setup
- Step: Define output directories and helper formatting functions.
- Output: Reusable utilities for table generation.


## Simulation Summary Tables
- Step: Build model-comparison tables from simulation summaries.
- Save: CSV and formatted outputs for reporting.


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

OUT_SIM_SUMMARY = RESULTS_ROOT / 'summary_simulations'
OUT_SIM_SUMMARY.mkdir(parents=True, exist_ok=True)

# 1) Base fit summaries (already mean/std over seeds) from combined A/B + C/D
fit_ab = TABLES['sim_summary_ab_long'].copy()
fit_cd = TABLES['sim_summary_cd_long'].copy()
fit_all = pd.concat([fit_ab, fit_cd], ignore_index=True)
fit_all = fit_all[(fit_all['task'] == 'fit') & (fit_all['effect_type'].isin(['alpha', 'beta']))].copy()

# 2) Compute MAE for fit from per-seed fit tables (not in existing fit summary CSVs)
def _seed_fit_path(scenario: str, seed: int) -> Path:
    return SIM_ROOT / scenario / str(seed) / f'scen{scenario}_fit_seed{seed}.csv'

mae_rows = []
for scenario in ['A', 'B', 'C', 'D']:
    for seed in [0, 1, 2, 3, 4]:
        fp = _seed_fit_path(scenario, seed)
        if not fp.exists():
            continue
        d = pd.read_csv(fp)
        d = d[d['effect_type'].isin(['alpha', 'beta'])].copy()
        d['ae'] = (d['mean'] - d['truth']).abs()
        g = d.groupby(['model', 'effect_type'], as_index=False)['ae'].mean()
        g = g.rename(columns={'ae': 'value', 'effect_type': 'effect_type'})
        g['scenario'] = scenario
        g['task'] = 'fit'
        g['metric'] = 'MAE'
        g['seed'] = seed
        mae_rows.append(g[['scenario', 'task', 'model', 'effect_type', 'metric', 'value', 'seed']])

if mae_rows:
    mae_seed = pd.concat(mae_rows, ignore_index=True)
    mae_summary = mae_seed.groupby(['scenario', 'task', 'model', 'effect_type', 'metric'], as_index=False)['value'].agg(mean='mean', std='std')
    fit_all = pd.concat([fit_all, mae_summary], ignore_index=True)

# 3) Keep requested metrics and format mean/std strings
wanted_metrics = ['MAE', 'RMSE', 'Coverage', 'CI_Width']
tab1 = fit_all[fit_all['metric'].isin(wanted_metrics)].copy()

tab1['metric_stat'] = tab1['metric'].astype(str) + '_mean_std'
tab1['value'] = tab1.apply(lambda r: f"{r['mean']:.4f} ({r['std']:.4f})", axis=1)

tab1_wide = (
    tab1.pivot_table(
        index=['model', 'scenario', 'effect_type'],
        columns='metric_stat',
        values='value',
        aggfunc='first'
    )
    .reset_index()
)

# enforce order
model_order = ['FE+AR', 'GP-CP', 'GP-CP-Extended']
scenario_order = ['A', 'B', 'C', 'D']
effect_order = ['alpha', 'beta']

tab1_wide['model'] = pd.Categorical(tab1_wide['model'], categories=model_order, ordered=True)
tab1_wide['scenario'] = pd.Categorical(tab1_wide['scenario'], categories=scenario_order, ordered=True)
tab1_wide['effect_type'] = pd.Categorical(tab1_wide['effect_type'], categories=effect_order, ordered=True)

tab1_wide = tab1_wide.sort_values(['model', 'scenario', 'effect_type']).reset_index(drop=True)

# 4) Print subtables by model (rows: scenario x effect)
subtables = {}
for m in model_order:
    sub = tab1_wide[tab1_wide['model'] == m].copy()
    sub = sub.drop(columns=['model'])
    subtables[m] = sub
    print(f"\n=== Table 1 Subtable: {m} ===")
    display(sub)

# 5) Save outputs
(tab1_wide).to_csv(OUT_SIM_SUMMARY / 'table1_model_fitting_comparison_all_models.csv', index=False)
for m, sub in subtables.items():
    safe = m.replace('+', 'plus').replace('-', '_').lower()
    sub.to_csv(OUT_SIM_SUMMARY / f'table1_model_fitting_comparison_{safe}.csv', index=False)

print('Wrote:', OUT_SIM_SUMMARY / 'table1_model_fitting_comparison_all_models.csv')
for m in model_order:
    safe = m.replace('+', 'plus').replace('-', '_').lower()
    print('Wrote:', OUT_SIM_SUMMARY / f'table1_model_fitting_comparison_{safe}.csv')

### Extrapolation Table
- Step: Summarize extrapolation performance metrics by model.
- Save: Final extrapolation comparison table.


In [ ]:
OUT_SIM_SUMMARY = RESULTS_ROOT / 'summary_simulations'
OUT_SIM_SUMMARY.mkdir(parents=True, exist_ok=True)

sim_ab = TABLES['sim_summary_ab_long'].copy()
sim_cd = TABLES['sim_summary_cd_long'].copy()
sim_all = pd.concat([sim_ab, sim_cd], ignore_index=True)

ex = sim_all[(sim_all['task'] == 'extrapolation') & (sim_all['effect_type'] == 'beta')].copy()
ex = ex[ex['metric'].isin(['MAE', 'RMSE', 'Coverage', 'CI_Width'])].copy()

ex['metric_col'] = ex['metric'].astype(str) + '_mean_std'
ex['value'] = ex.apply(lambda r: f"{r['mean']:.4f} ({r['std']:.4f})", axis=1)

table2 = (
    ex.pivot_table(
        index=['scenario', 'model'],
        columns='metric_col',
        values='value',
        aggfunc='first'
    )
    .reset_index()
)

scenario_order = ['A', 'B', 'C', 'D']
model_order = ['FE+AR', 'GP-CP', 'GP-CP-Extended']

table2['scenario'] = pd.Categorical(table2['scenario'], scenario_order, ordered=True)
table2['model'] = pd.Categorical(table2['model'], model_order, ordered=True)
table2 = table2.sort_values(['scenario', 'model']).reset_index(drop=True)

print('Table 2 preview:')
display(table2)

table2.to_csv(OUT_SIM_SUMMARY / 'table2_extrapolation_comparison_all_models.csv', index=False)
print('Wrote:', OUT_SIM_SUMMARY / 'table2_extrapolation_comparison_all_models.csv')

### Placebo Table
- Step: Summarize placebo metrics by model.
- Save: Final placebo comparison table.


In [ ]:
sim_ab = TABLES['sim_summary_ab_long'].copy()
sim_cd = TABLES['sim_summary_cd_long'].copy()
sim_all = pd.concat([sim_ab, sim_cd], ignore_index=True)

pl = sim_all[sim_all['task'] == 'placebo_te'].copy()

metric_map = {
    'tau_mae_avg_over_cohorts': ('tau_t', 'MAE'),
    'tau_rmse_avg_over_cohorts': ('tau_t', 'RMSE'),
    'tau_coverage_avg_over_cohorts': ('tau_t', 'Coverage'),
    'tau_ci_width_avg_over_cohorts': ('tau_t', 'CI_Width'),
    'att_mae_vs_zero': ('ATT', 'MAE'),
    'att_rmse_vs_zero': ('ATT', 'RMSE'),
    'att_coverage_vs_zero': ('ATT', 'Coverage'),
    'att_ci_width_avg_over_t': ('ATT', 'CI_Width'),
}

pl = pl[pl['metric'].isin(metric_map.keys())].copy()
pl['estimand'] = pl['metric'].map(lambda x: metric_map[x][0])
pl['metric_std'] = pl['metric'].map(lambda x: metric_map[x][1])
pl['metric_col'] = pl['metric_std'].astype(str) + '_mean_std'
pl['value'] = pl.apply(lambda r: f"{r['mean']:.4f} ({r['std']:.4f})", axis=1)

table3 = (
    pl.pivot_table(
        index=['scenario', 'estimand', 'model'],
        columns='metric_col',
        values='value',
        aggfunc='first'
    )
    .reset_index()
)

scenario_order = ['A', 'B', 'C', 'D']
estimand_order = ['tau_t', 'ATT']
model_order = ['FE+AR', 'GP-CP', 'GP-CP-Extended']

table3['scenario'] = pd.Categorical(table3['scenario'], scenario_order, ordered=True)
table3['estimand'] = pd.Categorical(table3['estimand'], estimand_order, ordered=True)
table3['model'] = pd.Categorical(table3['model'], model_order, ordered=True)
table3 = table3.sort_values(['scenario', 'estimand', 'model']).reset_index(drop=True)

print('Table 3 preview:')
display(table3)

table3.to_csv(OUT_SIM_SUMMARY / 'table3_placebo_comparison_all_models.csv', index=False)
print('Wrote:', OUT_SIM_SUMMARY / 'table3_placebo_comparison_all_models.csv')

## Application Summary Tables
- Step: Build real-data summary tables across categories.
- Save: Final application comparison outputs.


In [ ]:
OUT_APP_SUMMARY = RESULTS_ROOT / 'summary_applications'
OUT_APP_SUMMARY.mkdir(parents=True, exist_ok=True)

app_fit = TABLES['app_taskA_all_categories'].copy()

cat_map = {
    'gym_instore': 'Gym Instore',
    'mealkit': 'Mealkit',
    'streaming': 'Streaming Services',
}

table4 = app_fit[[
    'category', 'model', 'mae_fit', 'rmse_fit', 'coverage', 'ci_width_mean'
]].copy()

table4['category'] = table4['category'].map(cat_map).fillna(table4['category'])

table4 = table4.rename(columns={
    'category': 'Category',
    'model': 'Model',
    'mae_fit': 'MAE',
    'rmse_fit': 'RMSE',
    'coverage': 'Coverage',
    'ci_width_mean': 'CI_Width',
})

cat_order = ['Gym Instore', 'Mealkit', 'Streaming Services']
model_order = ['FE+AR', 'GP-CP', 'GP-CP-Extended']

table4['Category'] = pd.Categorical(table4['Category'], categories=cat_order, ordered=True)
table4['Model'] = pd.Categorical(table4['Model'], categories=model_order, ordered=True)

table4 = table4.sort_values(['Category', 'Model']).reset_index(drop=True)

for c in ['MAE', 'RMSE', 'Coverage', 'CI_Width']:
    table4[c] = table4[c].astype(float)

print('Table 4 preview:')
display(table4)

out_csv = OUT_APP_SUMMARY / 'table4_application_full_fitting_comparison.csv'
table4.to_csv(out_csv, index=False)
print('Wrote:', out_csv)